# Example of solute and water transport simulation
example from the documentation

In [12]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['../src'])

Python 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:38:13) [GCC 12.3.0] on linux


In [13]:
import math
from openalea.hydroroot import flux
from openalea.hydroroot.main import root_builder
from openalea.hydroroot.init_parameter import Parameters
from openalea.hydroroot.display import plot
from openalea.hydroroot.read_file import read_archi_data
from openalea.hydroroot.conductance import set_conductances
from openalea.hydroroot.water_solute_transport import pressure_calculation_no_non_permeating_solutes, init_some_MTG_properties
from oawidgets.plantgl import PlantGL
from openalea.hydroroot.display import mtg_scene

# for the PlantGL viewer used in hydroroot.display.plot
%gui qt 

Read the yaml file and set the Parameters variables, assuming that the code is run from the example folder

In [14]:
parameter = Parameters()
parameter.read_file('parameters_Ctr-3P2.yml')

In the code the concentration are in $mol.\mu L^{-1}$

In [15]:
Cse = parameter.solute['Cse'] * 1e-9 # mol/m3 -> mol/microL, external permeating solute concentration
Ce = parameter.solute['Ce'] * 1e-9 # mol/m3 -> mol/microL, external non-permeating solute concentration

Read the architecture file and build the MTG

In [16]:
fname = parameter.archi['input_dir'] + parameter.archi['input_file'][0]
df = read_archi_data(fname)
g, primary_length, total_length, surface, seed = root_builder( primary_length = parameter.archi['primary_length'],
                                                                delta = parameter.archi['branching_delay'],
                                                                nude_length = parameter.archi['nude_length'], 
                                                                df = df,
                                                                segment_length = parameter.archi['segment_length'],
                                                                length_data = parameter.archi['length_data'],
                                                                order_max = parameter.archi['order_max'],
                                                                order_decrease_factor = parameter.archi['order_decrease_factor'],
                                                                ref_radius = parameter.archi['ref_radius'])

Set the conductance in the MTG (in previous examples that was done in hydroroot_flow), set some other properties in *init_some_MTG_properties* and perform some initialization. Note that here *parameter.hydro['k0']* is a float.

In [17]:
g = set_conductances(g, axial_pr = parameter.hydro['axial_conductance_data'], k0_pr = parameter.hydro['k0']) 
g = flux.flux(g, psi_e = parameter.exp['psi_e'], psi_base = parameter.exp['psi_base'])  # initialization
g = init_some_MTG_properties(g, tau = parameter.solute['J_s'], Cini = parameter.solute['Cse'])

Perform the calculation, this a Newtown-Raphson loop on a matrix system. *pressure_calculation_no_non_permeating_solutes*, as its name indicates, is a solving function where no non-permeating solute is considered inside the root.

In [18]:
eps = 1.0e-9 # global: stop criterion for the Newton-Raphson loop in Jv_P_calculation and Jv_cnf_calculation
nb_v = g.nb_vertices()
Fdx = 1.0
Fdx_old = 1.
while Fdx > eps:
    g, dx, data, row, col = pressure_calculation_no_non_permeating_solutes(g, sigma = parameter.solute['Sigma'], 
                                                                           Ce = Ce,
                                                                           Cse = Cse, 
                                                                           Pe = parameter.exp['psi_e'], 
                                                                           Pbase = parameter.exp['psi_base'])
    Fdx = math.sqrt(sum(dx ** 2.0)) / nb_v
    if abs(Fdx - Fdx_old) < eps: break
    Fdx_old = Fdx
Jv = g.property('J_out')[1]

In [19]:
result=f"""
primary length (m): {primary_length}
surface (m2): {surface}
total length (m): {total_length}
flux (microL/s): {Jv}
"""
print(result)


primary length (m): 0.434
surface (m2): 0.005643500494241343
total length (m): 3.979
flux (microL/s): 0.0274121270124096



Display the concentration in the architecture

In [20]:
s = mtg_scene(g, prop_cmap = 'C') # create a scene from the mtg with the property j is the radial flux in ul/s
PlantGL(s) # display the root into the plantgl oawidget

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…